# Training procedure: 3PM-Noise2Void

In [1]:
# Jupyter-notebook magic

# For the matplotlib 
%matplotlib inline
# For reload functions explicitly
%load_ext autoreload
%autoreload 2
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# Imports

## Add the modules to the system path
import os
import sys
sys.path.append(os.path.join(".."))

## Libs
from random import shuffle
import glob
import tifffile
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
cudnn.fastest = True

## Own modules
import utils
from train import Trainer

## Channel selection

In [3]:
# Select here, if mGFP or THG should be denoised #
denoise_mGFP = True
#************************************************#

## Folder Structures

In [4]:
# Enter the store path for the results and raw file here #
path_results = os.path.join("..", "results_3PM-N2V")
if denoise_mGFP:
    path_results = path_results+"_mGFP"
else:
    path_results = path_results+"_THG"

path_dataset = os.path.join("..", "data", "3PM-N2V")
#********************************************************#

In [5]:
# Create all the other paths based on the results folder

# Make a folder to store results
res_folder = os.path.join(path_results, 'training_results')
os.makedirs(res_folder, exist_ok=True)

# Make a folder to store the log files
log_folder = os.path.join(path_results, 'log_files')
os.makedirs(log_folder, exist_ok=True)

log_train_folder = os.path.join(log_folder, 'train')
os.makedirs(log_train_folder, exist_ok=True)

log_val_folder = os.path.join(log_folder, 'val')
os.makedirs(log_val_folder, exist_ok=True)

# Make a folder for the checkpoints
checkpoint_folder = os.path.join(path_results, 'checkpoints')
os.makedirs(checkpoint_folder, exist_ok=True)

# List all folders in the results folder to make sure all folder exists
output_files = os.listdir(path_results)
print("*****Output Folder*****")
print("List of all folder in the results path:")
print(output_files)
print("***********************")

*****Output Folder*****
List of all folder in the results path:
['checkpoints', 'log_files', 'training_results']
***********************


## Parameters

### Load datasets

In [6]:
## Load image stack as dataset 

filenames_train = glob.glob(os.path.join(path_dataset, "*-train.tif"))
filenames_val = glob.glob(os.path.join(path_dataset, "*-validate.tif"))
print("On following file will be trained:  ", filenames_train[0])
print("on following file will be validated:  ", filenames_val[0])

file_train = tifffile.imread(filenames_train[0])
file_val = tifffile.imread(filenames_val[0])

if denoise_mGFP:
    file_train = file_train[:,0].squeeze()
    file_val = file_val[:,0].squeeze()
else:
    file_val = file_train[:,1].squeeze()
    file_val = file_train[:,1].squeeze()

On following file will be trained:   ..\data\3PM-N2V\RD5_3P_data-train.tif
on following file will be validated:   ..\data\3PM-N2V\RD5_3P_data-validate.tif


### Noise2Void Parameters

In [7]:
# Select the blindspot parameters #
percent_blindpixel = 2
picking_radius = 5
use_N2V2_setup = True
#*********************************#

### Training parameters

In [8]:
# Select the training parameters #
# Z x Y x X
input_size = [16, 64, 64]

# #Training-to-#Validation ratio
train_val_fraction = 0.5

# Training epochs
epoch = 300

# Batch size
batch_size = 4

# Logger frequencies
display_freq = 500
model_storing_freq = 50
#*********************************#

## Parameter Dictionary

In [9]:
# Parameter dictionary
parameter_dict= {}
# paths
# In case norm-factors are stored somewhere, not necessary
parameter_dict['dir_norm_factors'] = os.path.join("no_norm_factors_stored")
parameter_dict['dir_checkpoint'] = checkpoint_folder
parameter_dict['dir_log'] = log_folder
parameter_dict['dir_result'] = res_folder
# training state
parameter_dict['train_continue'] = 'on'
# hyperparameters
parameter_dict['num_epoch'] = epoch
# batch size
parameter_dict['batch_size'] = batch_size
# adam optimizer
parameter_dict['lr'] = 0.001
parameter_dict['optim'] = 'adam'
parameter_dict['beta1'] = 0.5
parameter_dict['beta2'] = 0.999
# colormap
parameter_dict['cmap'] = 'gray'
# size of the input patches
parameter_dict['ny'] = input_size[2]
parameter_dict['nx'] = input_size[1]
parameter_dict['nz'] = input_size[0]
# channel dimension
parameter_dict['nch'] = 1

# augmentation data for the N2V augmenter
parameter_dict['perc_pixel'] = percent_blindpixel
parameter_dict['n2v_neighborhood_radius'] = picking_radius
parameter_dict['structN2Vmask'] = None
# Use the N2V2 setup
parameter_dict['N2V2'] = use_N2V2_setup
# logger parameter
parameter_dict['num_freq_disp'] = display_freq
parameter_dict['num_freq_save'] = model_storing_freq
# datasets
parameter_dict['train_dataset'] = [file_train]
parameter_dict['val_dataset'] = [file_val[:-int(train_val_fraction*len(file_train))]]

## Training

In [10]:
# Show the parameters
print("***** Parameters *****")
print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in parameter_dict.items()) + "}")
print("**********************")

***** Parameters *****
{'dir_norm_factors': 'no_norm_factors_stored',
'dir_checkpoint': '..\\results_3PM-N2V_mGFP\\checkpoints',
'dir_log': '..\\results_3PM-N2V_mGFP\\log_files',
'dir_result': '..\\results_3PM-N2V_mGFP\\training_results',
'train_continue': 'on',
'num_epoch': 300,
'batch_size': 4,
'lr': 0.001,
'optim': 'adam',
'beta1': 0.5,
'beta2': 0.999,
'cmap': 'gray',
'ny': 64,
'nx': 64,
'nz': 16,
'nch': 1,
'perc_pixel': 2,
'n2v_neighborhood_radius': 5,
'structN2Vmask': None,
'num_freq_disp': 500,
'num_freq_save': 50,
'train_dataset': [array([[[32884, 32946, 32958, ..., 32889, 32927, 32897],
        [32985, 32886, 32890, ..., 32901, 32938, 33022],
        [32888, 32929, 32945, ..., 32990, 33020, 33272],
        ...,
        [33020, 32958, 33004, ..., 33020, 33168, 32984],
        [32931, 32930, 32892, ..., 33039, 32994, 32893],
        [32902, 32953, 32966, ..., 32959, 32914, 32912]],

       [[32905, 32888, 32934, ..., 32978, 32948, 32907],
        [32902, 32935, 32904, ..., 33007,

In [11]:
# Tensorboard logger
%tensorboard --logdir train:/../results_mGFP/log_files/train validation:/../results_mGFP/log_files/val

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
            

In [12]:
# Generate Trainer
trainer = Trainer(parameter_dict)
# Start training
print("*****Start of Training*****")
trainer.train()
print("*****End of Training*******")

GPU is available
*****Start of Training*****
Minimum: 0.5016708628976883	 Maximum: 0.5051346608682383
Minimum: 0.501686121919585	 Maximum: 0.5057145037003128
Number of blindspots:     1310
Number of blindspots:     1310
initialize network with normal
TRAIN: EPOCH 1: BATCH 0001/0008: LOSS1: 0.0006: LOSS2: 0.0002: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0002/0008: LOSS1: 0.0006: LOSS2: 0.0004: LOSS_TOTAL: 0.0005
TRAIN: EPOCH 1: BATCH 0003/0008: LOSS1: 0.0006: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0004/0008: LOSS1: 0.0006: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0005/0008: LOSS1: 0.0005: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0006/0008: LOSS1: 0.0005: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0007/0008: LOSS1: 0.0005: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
TRAIN: EPOCH 1: BATCH 0008/0008: LOSS1: 0.0005: LOSS2: 0.0003: LOSS_TOTAL: 0.0004
VALID: EPOCH 1: BATCH 0001/0004: LOSS1: 0.0004: LOSS2: 0.0001: LOSS_TOTAL: 0.0002
VALID: EPOC